### 문제
- 문자열 $s$ 가 주어졌을 때, $s$ 의 left cyclic shift 로 array를 만든다고 할때, 그 array를 $S$ 라고 하자. \
그리고 $S$ 사전순 정렬한 array $S'$ 의 첫 번째 원소를 $a'_1$ 이라고 하자, $a'_1$ 은 $S$ 의 몇 번째 원소인지 출력하는 문제이다.
- 출력이 0-index 이다.
- $L \le 10^5$ 이므로 $t$ 가 적당히 높을 것을 대비해 쿼리당 $O(N)$ 으로 풀어야 할 것이다

### 아이디어
- Suffix Array 느낌이 물씬 들었다. 그쪽 관련해서 생각해보기로 했다.
- 13322(접두사 배열) 의 아이디어에서 접두사 뒤에 S의 다른 문자를 추가하면 사전순으로 무조건 뒤에 온다는 사실을 이용할 수 있을 것 같았다.
  - S를 2번 이어 붙인 후, Suffix array를 구하고 나서, 그 배열의 값이 L보다 작은 것들을 찾는다면, 원하는 것을 구할 수 있을 것 같았다.
    - 우선 'baabaa' 를 예시로 생각해보자. 이를 2번 이어 붙인다면 'baabaabaabaa' 가 된다. 이를 Suffix Array 로 만들면 다음과 같다.\
    `[11, 10, 7, 4, 1, 8, 5, 2, 9, 6, 3, 0]` \
    여기서 원본 문자열보다 긴 것들은 별로 의미가 없으므로 제거한다면 다음과 같이 된다. \
    `[4, 1, 5, 2, 3, 0]`\
    첫 번째 원소는 4이지만, 정답은 1이다. 하지만 cyclic shift상의 4번째 문자열부터 시작한 것과 1번째부터 시작한 문자열은 `aabaab`로 같으니까 이 부분을 처리해야 한다.\
    그렇다면 첫번째 원소와 같은 문자열이면서, 가장 작은 position을 찾는 방법을 생각해보자.
    

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def SAIS(A):
  n = len(A)
  buckets = [0] * (max(A) + 2)
  for a in A:
    buckets[a + 1] += 1
  for b in range(1, len(buckets)):
    buckets[b] += buckets[b - 1]
  isL = [1] * n
  for i in reversed(range(n - 1)):
    isL[i] = +(A[i] > A[i + 1]) if A[i] != A[i + 1] else isL[i + 1]

  def induced_sort(LMS):
    SA = [-1] * (n)
    SA.append(n)
    endpoint = buckets[1:]
    for j in reversed(LMS):
      endpoint[A[j]] -= 1
      SA[endpoint[A[j]]] = j
    startpoint = buckets[:-1]
    for i in range(-1, n):
      j = SA[i] - 1
      if j >= 0 and isL[j]:
        SA[startpoint[A[j]]] = j
        startpoint[A[j]] += 1
    SA.pop()
    endpoint = buckets[1:]
    for i in reversed(range(n)):
      j = SA[i] - 1
      if j >= 0 and not isL[j]:
        endpoint[A[j]] -= 1
        SA[endpoint[A[j]]] = j
    return SA

  isLMS = [+(i and isL[i - 1] and not isL[i]) for i in range(n)]
  isLMS.append(1)
  LMS = [i for i in range(n) if isLMS[i]]
  if len(LMS) > 1:
    SA = induced_sort(LMS)
    LMS2 = [i for i in SA if isLMS[i]]
    prev = -1
    j = 0
    for i in LMS2:
      i1 = prev
      i2 = i
      while prev >= 0 and A[i1] == A[i2]:
        i1 += 1
        i2 += 1
        if isLMS[i1] or isLMS[i2]:
          j -= isLMS[i1] and isLMS[i2]
          break
      j += 1
      prev = i
      SA[i] = j
    LMS = [LMS[i] for i in SAIS([SA[i] for i in LMS])]
  return induced_sort(LMS)

def KASAI(A, SA):
  n = len(A)
  rank = [0] * n
  for i in range(n):
    rank[SA[i]] = i
  LCP = [0] * (n - 1)
  k = 0
  for i in range(n):
    SAind = rank[i]
    if SAind == n - 1:
      continue
    j = SA[SAind + 1]
    while i + k < n and A[i + k] == A[j + k]:
      k += 1
    LCP[SAind] = k
    k -= k > 0
  return LCP

def sol() :
  N, S = input().split()
  SA = SAIS(S + S)
  LCP = KASAI(S + S, SA) + [0]
  A = [(i, sa) for i, sa in enumerate(SA) if sa < int(N)]
  A.reverse()
  i, ans = A.pop()
  while A : #같은 모양이면서 사전순으로 먼저 오는 것을 찾는다.
    if LCP[i] <= int(N) : #LCP가 N보다 작으면 같은 모양이 아니다.
      break
    i, sa = A.pop()
    ans = min(ans, sa)
  print(ans)

for _ in range(int(input())) :
  sol()

### 풀이
- 이전까지의 아이디어도 맞았고, 마지막에서 고민하던 부분도 아이디어가 맞았다. 하지만 증명을 하지 못하고 결국 답지를 보게 됐다.
  - LCP[i] 값이 L보다 크다면, 그 뒤의 문자열, 즉 2배로 합치기 전의 원본 문자열 $S$ 를 전부 공통 접미사로 두었다는 뜻이다.\
  즉, cyclic left으로 했을 때 같은 문자열이라는 것이 된다. 따라서 LCP[i]가 $L$ 보다 작아질 때 까지 최솟값을 갱신시키면 된다.